In [3]:
import pandas as pd
import json
import glob
import sys
import os
from tqdm import tqdm

In [51]:
def issue_paper_ids():
    """
    Paper ID generator
    """
    paper_id = 0
    while True:
        yield paper_id
        paper_id = paper_id + 1


def extract_from_json(
    filename,
    paper_id_generator,
    focal_field_list=[
        "issn",
        "authors",
        "cord_paper_id",
        "paper_id",
    ],
):
    """
    Extract the bibliography from the json file
    """
    with open(filename, "r") as f:
        data = json.load(f)

    paper = {f: None for f in focal_field_list}
    paper["cord_paper_id"] = data["paper_id"]
    paper["paper_id"] = next(paper_id_generator) 
    paper["authors"] = data["metadata"]["authors"] 

    raw_references = [
        v
        for k, v in data["bib_entries"].items()
        if ("BIBREF" in k) and v["year"] is not None
    ]  # Extract all references
    references = []
    for i, raw_ref in enumerate(raw_references):
        ref = {fi: raw_ref.get(fi, None) for fi in focal_field_list}
        doi = raw_ref["other_ids"].get("DOI", [None])
        ref["DOI"] = doi[0] if len(doi) > 0 else None
        references += [ref]

    for i, r in enumerate(references):
        references[i]["paper_id"] = next(paper_id_generator)

    return paper, references

In [48]:
os.chdir("/N/project/rcsc/raw_data/mag-2020-05-29/2020-06-08/")
os.getcwd()
meta = pd.read_csv('metadata.csv')

paper, references = extract_from_json(meta['pmc_json_files'].iloc[0], paper_id_generator)

In [50]:
meta

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137406,du5sskle,NaN,Medline; WHO,Lockdown: more domestic accidents than COVID-1...,10.1136/archdischild-2020-319547,NaN,32487724.0,unk,NaN,2020-06-02,"Bressan, Silvia; Gallo, Elisa; Tirelli, France...",Archives of disease in childhood,NaN,#505853,NaN,NaN,NaN,https://doi.org/10.1136/archdischild-2020-3195...,219285104.0
137407,scvbkmrq,eee5773e7b63469ced1d4e3c9200b37dc8190ed8,Medline; PMC,Crises drive innovation,10.1111/codi.15043,PMC7228363,32185863.0,no-cc,Crises often drive innovation. The COVID 19 pa...,2020-03-30,"Gunner, C. K.; Oliphant, R.; Watson, A. J. M.",Colorectal Dis,NaN,NaN,NaN,document_parses/pdf_json/eee5773e7b63469ced1d4...,document_parses/pmc_json/PMC7228363.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32185863/;...,212751992.0
137408,ie46gvqj,2f47b0b1d2688551d04874af0053cc6297466bd7,Medline; PMC,Improved expression of porcine epidemic diarrh...,10.1007/s11240-019-01562-1,PMC7089040,32214566.0,no-cc,The porcine epidemic diarrhea virus (PEDV) bel...,2019-02-02,"Tien, Nguyen-Quang-Duc; Huy, Nguyen-Xuan; Kim,...",Plant Cell Tissue Organ Cult,NaN,NaN,NaN,document_parses/pdf_json/2f47b0b1d2688551d0487...,document_parses/pmc_json/PMC7089040.xml.json,https://doi.org/10.1007/s11240-019-01562-1; ht...,59541089.0
137409,mjbuzl07,2183478af3f9343c0f566eee8b5d95a1abea6754,Medline; PMC,The First Human Epitope Map of the Alphaviral ...,10.1371/journal.pntd.0000739,PMC2903468,20644615.0,cc0,BACKGROUND: Venezuelan equine encephalitis vir...,2010-07-13,"Hunt, Ann R.; Frederickson, Shana; Maruyama, T...",PLoS Negl Trop Dis,NaN,NaN,NaN,document_parses/pdf_json/2183478af3f9343c0f566...,document_parses/pmc_json/PMC2903468.xml.json,https://

In [87]:
def extract_meta_json(filename):
    """
    Extract the metadata from the json file
    """
    try:
        with open(filename, "r") as f:
            data = json.load(f)
        return data["metadata"]["authors"]
    except OSError as e:
        return None

meta["pmcAuthors"] = meta['pmc_json_files'].dropna().apply(extract_meta_json)

In [94]:
pd.set_option('display.max_colwidth', 2)
meta['pdf_json_files']

0         document_parses/pdf_json/d1aafb70c066a2068b02786f8929fd9c900897fb.json
1         document_parses/pdf_json/6b0567729c2143a66d737eb0a2f63f2dce2e5a7d.json
2         document_parses/pdf_json/06ced00a5fc04215949aa72528f2eeaae1d58927.json
3         document_parses/pdf_json/348055649b6b8cf2b9a376498df9bf41f7123605.json
4         document_parses/pdf_json/5f48792a5fa08bed9f56016f4981ae2ca6031b32.json
                                           ...                                  
137406    NaN                                                                   
137407    document_parses/pdf_json/eee5773e7b63469ced1d4e3c9200b37dc8190ed8.json
137408    document_parses/pdf_json/2f47b0b1d2688551d04874af0053cc6297466bd7.json
137409    document_parses/pdf_json/2183478af3f9343c0f566eee8b5d95a1abea6754.json
137410    document_parses/pdf_json/6c17cfb2eb90e99b2ce3eff3d629965833b6c124.json
Name: pdf_json_files, Length: 137411, dtype: object

In [130]:
#pd.reset_option('display.max_colwidth')
meta

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,pmcAuthors,pdfAuthors,SSAuthors,pmcAffiliation
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,...,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,"[{'first': 'Tariq', 'middle': ['A'], 'last': '...","[{'first': 'Tariq', 'middle': ['A'], 'last': '...","[{'authorId': '2843307', 'name': 'Tariq A Mada...","[{}, {}]"
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,...,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Albert', 'middle': ['van', 'der'],...","[{'first': 'Albert', 'middle': [], 'last': 'Va...","[{'authorId': '89883601', 'name': 'Albert van ...","[{}, {}, {}]"
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,...,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Erika', 'middle': ['C'], 'last': '...","[{'first': 'Erika', 'middle': ['C'], 'last': '...","[{'authorId': '35112551', 'name': 'Erika C. Cr...",[{}]
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,...,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Karen', 'middle': ['A'], 'last': '...",[],"[{'authorId': '3913024', 'name': 'Karen A. Fag...","[{}, {}, {}]"
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,...,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Joseph', 'middle': ['B'], 'last': ...","[{'first': 'Joseph', 'middle': ['B'], 'last': ...","[{'authorId': '3563856', 'name': 'Joseph B. Do...","[{}, {}, {}]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137406,du5sskle,NaN,Medline; WHO,Lockdown: more domestic accidents than COVID-1...,10.1136/archdischild-2020-319547,NaN,32487724.0,unk,NaN,2020-06-02,...,#505853,NaN,NaN,NaN,https://doi.org/10.1136/archdischild-2020-3195...,219285104.0,NaN,NaN,NaN,NaN
137407,scvbkmrq,eee5773e7b63469ced1d4e3c9200b37dc8190ed8,Medline; PMC,Crises drive innovation,10.1111/codi.15043,PMC7228363,32185863.0,no-cc,Crises often drive innovation. The COVID 19 pa...,2020-03-30,...,NaN,NaN,document_parses/pdf_json/eee5773e7b63469ced1d4...,document_parses/pmc_json/PMC7228363.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32185863/;...,212751992.0,"[{'first': 'C.', 'middle': ['K.'], 'last': 'Gu...","[{'first': 'C', 'middle': ['K'], 'last': 'Gunn...",status Codes: 403,"[{}, {}, {}]"
137408,ie46gvqj,2f47b0b1d2688551d04874af0053cc6297466bd7,Medline; PMC,Improved expression of porcine epidemic diarrh...,10.1007/s11240-019-01562-1,PMC7089040,32214566.0,no-cc,The porcine epidemic diarrhea virus (PEDV) bel...,2019-02-02,...,NaN,NaN,document_parses/pdf_json/2f47b0b1d2688551d0487...,document_parses/pmc_json/PMC7089040.xml.json,ht

In [137]:
meta.to_csv("metaAuthors.csv")
#paper_refs.to_csv("papersPDFrefs.csv")

In [139]:
f = open('document_parses/pdf_json/6b0567729c2143a66d737eb0a2f63f2dce2e5a7d.json', "r")
json.load(f)

{'paper_id': '6b0567729c2143a66d737eb0a2f63f2dce2e5a7d',
 'metadata': {'title': 'Nitric oxide: a pro-inflammatory mediator in lung disease?',
  'authors': [{'first': 'Albert',
    'middle': [],
    'last': 'Van Der Vliet',
    'suffix': '',
    'affiliation': {},
    'email': ''},
   {'first': 'Jason',
    'middle': ['P'],
    'last': 'Eiserich',
    'suffix': '',
    'affiliation': {'laboratory': '',
     'institution': 'University of Alabama at Birmingham',
     'location': {'settlement': 'Birmingham',
      'region': 'Alabama',
      'country': 'USA'}},
    'email': ''},
   {'first': 'Carroll',
    'middle': ['E'],
    'last': 'Cross',
    'suffix': '',
    'affiliation': {},
    'email': ''}]},
 'abstract': [{'text': 'Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO • ) and increased indices of NO • -dependent oxidative stress. Although NO • is known to have anti-microbial, anti-inflammatory and anti-oxidant propert

In [138]:
import requests

resp = requests.get('https://api.semanticscholar.org/v1/paper/d1aafb70c066a2068b02786f8929fd9c900897fb')
if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))
#resp.json()['references']
resp.json()

{'abstract': 'ObjectiveThis retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia.MethodsPatients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed.Results40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) 

In [135]:
import requests
def get_SS_json(url):
    """
    Extract the metadata from SemanticScholar REST API
    """
    resp = requests.get('https://api.semanticscholar.org/v1/paper/'+url)
    if resp.status_code != 200:
        # This means something went wrong.
        #return None
        return ('status Codes: '+str(resp.status_code))
    else :    
        return resp.json()['references']

meta["SSRefs"] = meta['sha'].dropna().apply(get_SS_json)

In [126]:
meta['pmcAuthors'].iloc[0]

[{'first': 'Tariq',
  'middle': ['A'],
  'last': 'Madani',
  'suffix': '',
  'email': 'tmadani@health.net.sa',
  'affiliation': {}},
 {'first': 'Aisha',
  'middle': ['A'],
  'last': 'Al-Ghamdi',
  'suffix': '',
  'email': 'aalghamdius@yahoo.com',
  'affiliation': {}}]

In [129]:
def get_Affiliations(authorList):
    """
    Extract the metadata from the json file
    """
    AffiliationsList = [d.get('affiliation') for d in authorList]
    return AffiliationsList
#get_Affiliations(meta['pmcAuthors'].iloc[0])  
meta['pmcAffiliation'] = meta['pmcAuthors'].dropna().apply(get_Affiliations)

In [131]:
meta['pdfAffiliation'] = meta['pdfAuthors'].dropna().apply(get_Affiliations)
meta

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,pmcAuthors,pdfAuthors,SSAuthors,pmcAffiliation,pdfAffiliation
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,...,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,"[{'first': 'Tariq', 'middle': ['A'], 'last': '...","[{'first': 'Tariq', 'middle': ['A'], 'last': '...","[{'authorId': '2843307', 'name': 'Tariq A Mada...","[{}, {}]","[{}, {}]"
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,...,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Albert', 'middle': ['van', 'der'],...","[{'first': 'Albert', 'middle': [], 'last': 'Va...","[{'authorId': '89883601', 'name': 'Albert van ...","[{}, {}, {}]","[{}, {'laboratory': '', 'institution': 'Univer..."
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,...,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Erika', 'middle': ['C'], 'last': '...","[{'first': 'Erika', 'middle': ['C'], 'last': '...","[{'authorId': '35112551', 'name': 'Erika C. Cr...",[{}],"[{'laboratory': '', 'institution': 'Washington..."
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,...,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Karen', 'middle': ['A'], 'last': '...",[],"[{'authorId': '3913024', 'name': 'Karen A. Fag...","[{}, {}, {}]",[]
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,...,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Joseph', 'middle': ['B'], 'last': ...","[{'first': 'Joseph', 'middle': ['B'], 'last': ...","[{'authorId': '3563856', 'name': 'Joseph B. Do...","[{}, {}, {}]","[{}, {}, {'laboratory': 'Laboratory of Host De..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137406,du5sskle,NaN,Medline; WHO,Lockdown: more domestic accidents than COVID-1...,10.1136/archdischild-2020-319547,NaN,32487724.0,unk,NaN,2020-06-02,...,NaN,NaN,NaN,https://doi.org/10.1136/archdischild-2020-3195...,219285104.0,NaN,NaN,NaN,NaN,NaN
137407,scvbkmrq,eee5773e7b63469ced1d4e3c9200b37dc8190ed8,Medline; PMC,Crises drive innovation,10.1111/codi.15043,PMC7228363,32185863.0,no-cc,Crises often drive innovation. The COVID 19 pa...,2020-03-30,...,NaN,document_parses/pdf_json/eee5773e7b63469ced1d4...,document_parses/pmc_json/PMC7228363.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32185863/;...,212751992.0,"[{'first': 'C.', 'middle': ['K.'], 'last': 'Gu...","[{'first': 'C', 'middle': ['K'], 'last': 'Gunn...",status Codes: 403,"[{}, {}, {}]",[{'laboratory': '†West of Scotland Cancer Surv...
137408,ie46gvqj,2f47b0b1d2688551d04874af0053cc6297466bd7,Medline; PMC,Improved expression of porcine epidemic diarrh...,10.1007/s11240-019-01562-1,PMC70890

In [136]:
meta

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,pdf_json_files,pmc_json_files,url,s2_id,pmcAuthors,pdfAuthors,SSAuthors,pmcAffiliation,pdfAffiliation,SSRefs
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,...,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,"[{'first': 'Tariq', 'middle': ['A'], 'last': '...","[{'first': 'Tariq', 'middle': ['A'], 'last': '...","[{'authorId': '2843307', 'name': 'Tariq A Mada...","[{}, {}]","[{}, {}]","[{'arxivId': None, 'authors': [{'authorId': '4..."
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,...,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Albert', 'middle': ['van', 'der'],...","[{'first': 'Albert', 'middle': [], 'last': 'Va...","[{'authorId': '89883601', 'name': 'Albert van ...","[{}, {}, {}]","[{}, {'laboratory': '', 'institution': 'Univer...","[{'arxivId': None, 'authors': [{'authorId': '3..."
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,...,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Erika', 'middle': ['C'], 'last': '...","[{'first': 'Erika', 'middle': ['C'], 'last': '...","[{'authorId': '35112551', 'name': 'Erika C. Cr...",[{}],"[{'laboratory': '', 'institution': 'Washington...","[{'arxivId': None, 'authors': [{'authorId': '4..."
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,...,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Karen', 'middle': ['A'], 'last': '...",[],"[{'authorId': '3913024', 'name': 'Karen A. Fag...","[{}, {}, {}]",[],"[{'arxivId': None, 'authors': [{'authorId': '5..."
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,...,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[{'first': 'Joseph', 'middle': ['B'], 'last': ...","[{'first': 'Joseph', 'middle': ['B'], 'last': ...","[{'authorId': '3563856', 'name': 'Joseph B. Do...","[{}, {}, {}]","[{}, {}, {'laboratory': 'Laboratory of Host De...","[{'arxivId': None, 'authors': [{'authorId': '9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137406,du5sskle,NaN,Medline; WHO,Lockdown: more domestic accidents than COVID-1...,10.1136/archdischild-2020-319547,NaN,32487724.0,unk,NaN,2020-06-02,...,NaN,NaN,https://doi.org/10.1136/archdischild-2020-3195...,219285104.0,NaN,NaN,NaN,NaN,NaN,NaN
137407,scvbkmrq,eee5773e7b63469ced1d4e3c9200b37dc8190ed8,Medline; PMC,Crises drive innovation,10.1111/codi.15043,PMC7228363,32185863.0,no-cc,Crises often drive innovation. The COVID 19 pa...,2020-03-30,...,document_parses/pdf_json/eee5773e7b63469ced1d4...,document_parses/pmc_json/PMC7228363.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32185863/;...,212751992.0,"[{'first': 'C.', 'middle': ['K.'], 'last': 'Gu...","[{'first': 'C', 'middle': ['K'], 'last': 'Gunn...",stat

In [40]:
paper_id_generator = issue_paper_ids()
paper_refs = []
papers = []
for filename in tqdm(FILE_LIST):

    # Extract bibliography from json
    paper, references = extract_from_json(filename, paper_id_generator)

    # Record the citations between the paper and its references
    paper_refs += [(paper["paper_id"], ref["paper_id"]) for ref in references]

    # Record extracted papers
    papers += [pd.DataFrame([paper] + references)]

# Pack into the pandas DataFrame
papers = pd.concat(papers, ignore_index=True)
paper_refs = pd.DataFrame(paper_refs, columns=["source", "target"])

# Save
papers.to_csv("papersPMC.csv")
paper_refs.to_csv("papersPMCrefs.csv")



  0%|          | 0/50801 [00:00<?, ?it/s]

  0%|          | 3/50801 [00:00<32:53, 25.74it/s]

  0%|          | 6/50801 [00:00<33:09, 25.53it/s]

  0%|          | 8/50801 [00:00<35:57, 23.55it/s]

  0%|          | 12/50801 [00:00<33:36, 25.18it/s]

  0%|          | 15/50801 [00:00<32:02, 26.41it/s]

  0%|          | 18/50801 [00:00<31:41, 26.71it/s]

  0%|          | 21/50801 [00:00<33:13, 25.47it/s]

  0%|          | 26/50801 [00:00<28:29, 29.70it/s]

  0%|          | 32/50801 [00:01<25:33, 33.10it/s]

  0%|          | 38/50801 [00:01<22:22, 37.80it/s]

  0%|          | 43/50801 [00:01<25:42, 32.90it/s]

  0%|          | 47/50801 [00:01<25:02, 33.77it/s]

  0%|          | 51/50801 [00:01<30:43, 27.53it/s]

  0%|          | 55/50801 [00:01<30:17, 27.91it/s]

  0%|          | 59/50801 [00:01<29:02, 29.12it/s]

  0%|          | 63/50801 [00:02<28:21, 29.83it/s]

  0%|          | 67/50801 [00:02<27:04, 31.23it/s]

  0%|          | 71/50801 [00:02<28:53, 29.26it/s]

  0%|          | 75/50

  1%|          | 562/50801 [00:18<27:51, 30.06it/s]

  1%|          | 567/50801 [00:18<25:08, 33.30it/s]

  1%|          | 571/50801 [00:18<24:07, 34.70it/s]

  1%|          | 575/50801 [00:18<23:18, 35.90it/s]

  1%|          | 579/50801 [00:18<25:26, 32.89it/s]

  1%|          | 583/50801 [00:19<29:33, 28.32it/s]

  1%|          | 587/50801 [00:19<29:41, 28.19it/s]

  1%|          | 591/50801 [00:19<27:43, 30.19it/s]

  1%|          | 595/50801 [00:19<26:19, 31.78it/s]

  1%|          | 599/50801 [00:19<25:40, 32.58it/s]

  1%|          | 603/50801 [00:19<25:59, 32.18it/s]

  1%|          | 607/50801 [00:19<26:24, 31.68it/s]

  1%|          | 611/50801 [00:20<28:09, 29.70it/s]

  1%|          | 615/50801 [00:20<30:21, 27.55it/s]

  1%|          | 619/50801 [00:20<29:33, 28.29it/s]

  1%|          | 624/50801 [00:20<27:35, 30.31it/s]

  1%|          | 628/50801 [00:20<29:06, 28.73it/s]

  1%|          | 632/50801 [00:20<27:18, 30.63it/s]

  1%|▏         | 636/50801 [00:20<30:45, 27.19

  2%|▏         | 1118/50801 [00:36<29:12, 28.36it/s]

  2%|▏         | 1121/50801 [00:36<28:49, 28.73it/s]

  2%|▏         | 1125/50801 [00:36<28:37, 28.92it/s]

  2%|▏         | 1128/50801 [00:36<29:32, 28.02it/s]

  2%|▏         | 1132/50801 [00:37<27:42, 29.87it/s]

  2%|▏         | 1136/50801 [00:37<29:57, 27.62it/s]

  2%|▏         | 1140/50801 [00:37<27:48, 29.76it/s]

  2%|▏         | 1144/50801 [00:37<28:59, 28.54it/s]

  2%|▏         | 1147/50801 [00:37<32:02, 25.83it/s]

  2%|▏         | 1151/50801 [00:37<28:53, 28.64it/s]

  2%|▏         | 1155/50801 [00:37<29:02, 28.49it/s]

  2%|▏         | 1159/50801 [00:38<30:58, 26.71it/s]

  2%|▏         | 1163/50801 [00:38<28:57, 28.56it/s]

  2%|▏         | 1166/50801 [00:38<28:39, 28.86it/s]

  2%|▏         | 1169/50801 [00:38<29:59, 27.58it/s]

  2%|▏         | 1172/50801 [00:38<30:43, 26.92it/s]

  2%|▏         | 1177/50801 [00:38<28:44, 28.78it/s]

  2%|▏         | 1181/50801 [00:38<27:43, 29.82it/s]

  2%|▏         | 1185/50801 

  3%|▎         | 1675/50801 [00:54<24:29, 33.44it/s]

  3%|▎         | 1679/50801 [00:54<26:26, 30.97it/s]

  3%|▎         | 1684/50801 [00:54<24:33, 33.34it/s]

  3%|▎         | 1688/50801 [00:55<23:38, 34.62it/s]

  3%|▎         | 1694/50801 [00:55<20:54, 39.15it/s]

  3%|▎         | 1699/50801 [00:55<23:54, 34.24it/s]

  3%|▎         | 1703/50801 [00:55<26:02, 31.42it/s]

  3%|▎         | 1707/50801 [00:55<26:12, 31.22it/s]

  3%|▎         | 1711/50801 [00:55<26:25, 30.96it/s]

  3%|▎         | 1715/50801 [00:55<26:22, 31.01it/s]

  3%|▎         | 1721/50801 [00:56<23:14, 35.20it/s]

  3%|▎         | 1725/50801 [00:56<24:06, 33.93it/s]

  3%|▎         | 1732/50801 [00:56<20:44, 39.43it/s]

  3%|▎         | 1737/50801 [00:56<24:12, 33.77it/s]

  3%|▎         | 1741/50801 [00:56<25:15, 32.37it/s]

  3%|▎         | 1746/50801 [00:56<23:40, 34.54it/s]

  3%|▎         | 1750/50801 [00:56<23:41, 34.51it/s]

  3%|▎         | 1754/50801 [00:56<23:31, 34.74it/s]

  3%|▎         | 1758/50801 

  4%|▍         | 2274/50801 [01:12<24:36, 32.86it/s]

  4%|▍         | 2278/50801 [01:12<24:41, 32.74it/s]

  4%|▍         | 2282/50801 [01:13<24:38, 32.81it/s]

  4%|▍         | 2286/50801 [01:13<26:26, 30.57it/s]

  5%|▍         | 2291/50801 [01:13<24:49, 32.56it/s]

  5%|▍         | 2295/50801 [01:13<27:17, 29.63it/s]

  5%|▍         | 2299/50801 [01:13<25:47, 31.33it/s]

  5%|▍         | 2305/50801 [01:13<22:12, 36.41it/s]

  5%|▍         | 2310/50801 [01:13<21:01, 38.43it/s]

  5%|▍         | 2315/50801 [01:13<20:35, 39.25it/s]

  5%|▍         | 2320/50801 [01:14<21:32, 37.50it/s]

  5%|▍         | 2324/50801 [01:14<23:17, 34.68it/s]

  5%|▍         | 2328/50801 [01:14<24:25, 33.07it/s]

  5%|▍         | 2333/50801 [01:14<22:00, 36.69it/s]

  5%|▍         | 2337/50801 [01:14<23:14, 34.75it/s]

  5%|▍         | 2342/50801 [01:14<23:04, 34.99it/s]

  5%|▍         | 2346/50801 [01:14<25:58, 31.09it/s]

  5%|▍         | 2350/50801 [01:14<25:06, 32.15it/s]

  5%|▍         | 2354/50801 

...

 96%|█████████▌| 48578/50801 [29:02<01:11, 31.11it/s]

 96%|█████████▌| 48582/50801 [29:02<01:06, 33.13it/s]

 96%|█████████▌| 48586/50801 [29:02<01:04, 34.29it/s]

 96%|█████████▌| 48590/50801 [29:02<01:06, 33.48it/s]

 96%|█████████▌| 48594/50801 [29:03<01:06, 33.15it/s]

 96%|█████████▌| 48598/50801 [29:03<01:06, 33.35it/s]

 96%|█████████▌| 48603/50801 [29:03<01:02, 35.11it/s]

 96%|█████████▌| 48607/50801 [29:03<01:02, 34.89it/s]

 96%|█████████▌| 48611/50801 [29:03<01:08, 32.00it/s]

 96%|█████████▌| 48616/50801 [29:03<01:04, 33.75it/s]

 96%|█████████▌| 48620/50801 [29:03<01:03, 34.14it/s]

 96%|█████████▌| 48624/50801 [29:03<01:08, 31.67it/s]

 96%|█████████▌| 48628/50801 [29:04<01:11, 30.52it/s]

 96%|█████████▌| 48632/50801 [29:04<01:12, 29.77it/s]

 96%|█████████▌| 48636/50801 [29:04<01:12, 29.68it/s]

 96%|█████████▌| 48640/50801 [29:04<01:08, 31.34it/s]

 96%|█████████▌| 48644/50801 [29:04<01:08, 31.56it/s]

 96%|█████████▌| 48648/50801 [29:04<01:09, 30.97it/s]

 96%|█████

 97%|█████████▋| 49094/50801 [29:19<00:55, 30.82it/s]

 97%|█████████▋| 49098/50801 [29:19<00:57, 29.78it/s]

 97%|█████████▋| 49102/50801 [29:19<00:55, 30.40it/s]

 97%|█████████▋| 49106/50801 [29:19<00:54, 31.02it/s]

 97%|█████████▋| 49110/50801 [29:20<00:59, 28.61it/s]

 97%|█████████▋| 49114/50801 [29:20<00:57, 29.33it/s]

 97%|█████████▋| 49117/50801 [29:20<01:00, 27.94it/s]

 97%|█████████▋| 49120/50801 [29:20<01:01, 27.29it/s]

 97%|█████████▋| 49124/50801 [29:20<01:00, 27.95it/s]

 97%|█████████▋| 49127/50801 [29:20<00:59, 28.26it/s]

 97%|█████████▋| 49130/50801 [29:20<00:59, 28.03it/s]

 97%|█████████▋| 49134/50801 [29:20<00:58, 28.59it/s]

 97%|█████████▋| 49138/50801 [29:21<00:56, 29.59it/s]

 97%|█████████▋| 49142/50801 [29:21<00:54, 30.30it/s]

 97%|█████████▋| 49146/50801 [29:21<00:54, 30.56it/s]

 97%|█████████▋| 49150/50801 [29:21<00:52, 31.23it/s]

 97%|█████████▋| 49154/50801 [29:21<00:52, 31.62it/s]

 97%|█████████▋| 49158/50801 [29:21<00:53, 30.68it/s]

 97%|█████

 98%|█████████▊| 49614/50801 [29:36<00:35, 33.89it/s]

 98%|█████████▊| 49618/50801 [29:36<00:36, 32.08it/s]

 98%|█████████▊| 49622/50801 [29:37<00:38, 30.74it/s]

 98%|█████████▊| 49626/50801 [29:37<00:40, 28.80it/s]

 98%|█████████▊| 49629/50801 [29:37<00:43, 27.09it/s]

 98%|█████████▊| 49632/50801 [29:37<00:44, 26.52it/s]

 98%|█████████▊| 49635/50801 [29:37<00:45, 25.57it/s]

 98%|█████████▊| 49638/50801 [29:37<00:43, 26.58it/s]

 98%|█████████▊| 49641/50801 [29:37<00:44, 25.96it/s]

 98%|█████████▊| 49644/50801 [29:37<00:44, 26.15it/s]

 98%|█████████▊| 49647/50801 [29:38<00:43, 26.79it/s]

 98%|█████████▊| 49650/50801 [29:38<00:43, 26.65it/s]

 98%|█████████▊| 49653/50801 [29:38<00:42, 27.30it/s]

 98%|█████████▊| 49657/50801 [29:38<00:38, 29.69it/s]

 98%|█████████▊| 49661/50801 [29:38<00:39, 29.11it/s]

 98%|█████████▊| 49664/50801 [29:38<00:40, 28.20it/s]

 98%|█████████▊| 49667/50801 [29:38<00:40, 27.94it/s]

 98%|█████████▊| 49671/50801 [29:38<00:39, 28.53it/s]

 98%|█████

 99%|█████████▊| 50094/50801 [29:54<00:28, 24.71it/s]

 99%|█████████▊| 50097/50801 [29:54<00:28, 24.85it/s]

 99%|█████████▊| 50100/50801 [29:54<00:28, 24.49it/s]

 99%|█████████▊| 50104/50801 [29:54<00:26, 26.41it/s]

 99%|█████████▊| 50107/50801 [29:54<00:25, 26.86it/s]

 99%|█████████▊| 50112/50801 [29:54<00:22, 30.46it/s]

 99%|█████████▊| 50116/50801 [29:54<00:21, 31.41it/s]

 99%|█████████▊| 50120/50801 [29:54<00:21, 31.97it/s]

 99%|█████████▊| 50124/50801 [29:55<00:21, 31.71it/s]

 99%|█████████▊| 50128/50801 [29:55<00:21, 30.83it/s]

 99%|█████████▊| 50132/50801 [29:55<00:21, 31.53it/s]

 99%|█████████▊| 50136/50801 [29:55<00:22, 29.67it/s]

 99%|█████████▊| 50140/50801 [29:55<00:23, 27.66it/s]

 99%|█████████▊| 50143/50801 [29:55<00:24, 27.32it/s]

 99%|█████████▊| 50146/50801 [29:55<00:23, 27.74it/s]

 99%|█████████▊| 50150/50801 [29:55<00:21, 29.61it/s]

 99%|█████████▊| 50154/50801 [29:56<00:21, 30.61it/s]

 99%|█████████▊| 50158/50801 [29:56<00:21, 29.97it/s]

 99%|█████

100%|█████████▉| 50598/50801 [30:11<00:06, 33.09it/s]

100%|█████████▉| 50602/50801 [30:11<00:06, 31.28it/s]

100%|█████████▉| 50606/50801 [30:11<00:06, 32.38it/s]

100%|█████████▉| 50610/50801 [30:11<00:06, 30.79it/s]

100%|█████████▉| 50614/50801 [30:11<00:05, 31.88it/s]

100%|█████████▉| 50618/50801 [30:11<00:05, 32.65it/s]

100%|█████████▉| 50623/50801 [30:11<00:05, 33.91it/s]

100%|█████████▉| 50627/50801 [30:11<00:04, 35.38it/s]

100%|█████████▉| 50631/50801 [30:12<00:05, 33.58it/s]

100%|█████████▉| 50635/50801 [30:12<00:05, 29.70it/s]

100%|█████████▉| 50639/50801 [30:12<00:05, 27.33it/s]

100%|█████████▉| 50642/50801 [30:12<00:06, 25.98it/s]

100%|█████████▉| 50645/50801 [30:12<00:06, 25.82it/s]

100%|█████████▉| 50648/50801 [30:12<00:05, 25.60it/s]

100%|█████████▉| 50651/50801 [30:12<00:06, 24.90it/s]

100%|█████████▉| 50654/50801 [30:13<00:05, 26.02it/s]

100%|█████████▉| 50658/50801 [30:13<00:05, 27.53it/s]

100%|█████████▉| 50662/50801 [30:13<00:04, 30.25it/s]

100%|█████